## Libraries and data

In [1]:
import numpy as np
import numpy.linalg as LA

data = np.genfromtxt('./data/valores.csv', delimiter=',')

## Global parameters

In [3]:
l = 0.3
h = 0.5
dt = 0.25
t_end = 250
n_steps = int(t_end/dt)
init_sigma = np.zeros((5, 5))
init_sigma[3,3] = 10
init_sigma[4,4] = 10
init_state = np.zeros((5,1))

## Auxiliary Functions

In [40]:
def jacobian(X: np.array, u: np.array, which: str) -> np.array:
    '''
    Calculados analiticamente.
    '''    
    if which=='A':
        ddx = (-u[1]*np.sin(X[2])*dt)[0]
        ddy = (u[1]*np.cos(X[2])*dt)[0]
        J = [
            [1, 0, ddx, 0, 0],
            [0, 1, ddy, 0, 0],
            [0, 0, 1, 0, 0],
            [0, 0, 0, 1, 0],
            [0, 0, 0, 0, 1]
        ]
    elif which=='C':
        sin = np.sin(x_prev[2])[0]
        cos = np.cos(x_pre[2])[0]
        J = [
            [X[0]/np.sqrt(X[0]**2 + X[1]**2 + h**2), X[1]/np.sqrt(X[0]**2 + X[1]**2 + h**2), 0, 0, 0],
            [0, 0, -x_prev[3]*sin + x_prev[4]*cos, cos, sin],
            [0, 0, -x_prev[3]*cos - x_prev[4]*sin, -sin, cos]
        ]
        
    return np.array(J)

def F(X_prev: np.array, u_prev: np.array) -> np.array:
    '''
    Calculada analiticamente.
    '''
    return np.array([
        X_prev[0] + u_prev[0] * np.cos(X_prev[2]) * dt,
        X_prev[1] + u_prev[0] * np.sin(X_prev[2]) * dt,
        X_prev[2] + u_prev[0] * dt * np.tan(u_prev[1])/ l,
        X_prev[3],
        X_prev[4]
    ])

def Q(x_prev):
    sf, se = 0.25, 0.25
    srho = (h**2 + x_prev[0]**2 + x_prev[1]**2) / 20**2
    return np.array([
        [srho, 0, 0],
        [0, sf, 0],
        [0, 0, se]
    ])

def R(x_prev, u):
    cos = np.cos(x_prev[2])[0]
    sin = np.sin(x_prev[2])[0]
    M =  np.array([
        [cos, -sin, 0, 0, 0],
        [sin,  cos, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]
    ])
    
    sl = (dt * u[0] / 6)**2
    sr = (dt * u[0] / 12)**2
    stheta = (dt * u[0] / (8*l))**2
    
    # Matriz R no referencial local
    Rl = np.array([
        [sl, 0, 0, 0, 0],
        [0, sr, 0, 0, 0],
        [0, 0, stheta, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ])
    
    return M @ Rl @ M.T

def G(x_prev):
    sin = np.sin(x_prev[2])[0]
    cos = np.cos(x_pre[2])[0]    
    return np.array([
        [np.sqrt(x_prev[0]**2 + x_prev[1]**2 + h**2)],
        [( x_prev[3] * cos + x_prev[4] * sin)],
        [(-x_prev[3] * sin + x_prev[4] * cos)]
    ])

## EKF

Implementação do Filtro de Kalman extendido

In [41]:
class EKF:
    def __init__(self, f, g, x_est, Sigma_est, Q, R, data):
        '''EKF(f, h, x0, u0)

        x' = f(x, u)    state transition function
        z' = g(x)       observation function
        x0              initial state estimate
        Sigma0          initial cov matrix estimate
        Q               stochastic matrix for observation
        R               stochastic matrix for state'''
        self.f, self.g = f, g
        self.x = np.array(x_est)
        self.I = np.eye(len(x_est))
        self.Sigma = Sigma_est
        self.Q, self.R = Q, R
        self.u = data[:,:2]
        self.z = data[:,2:]
    
    def predict(self, t: int):
        self.x = self.f(self.x, self.u[t])
        A = jacobian(X=self.x, u=self.u[t], which='A')
        self.Sigma = (A @ self.Sigma @ A.T) + self.R(self.x, self.u[t])
#         print(self.Sigma)

    def update(self, t: int):
        x, Sigma, Q, I, g, z = self.x, self.Sigma, self.Q, self.I, self.g, self.z
        C = jacobian(X=x, u=z[t], which='C')
        y = z[t] - g(x)
        S = (C @ Sigma @ C.T) + Q(x)
        K = Sigma @ C.T @ LA.inv(S)
        self.x += K @ y
        self.Sigma = (I - K @ C) @ Sigma

## Questão 04

Como nessa etapa não são consideradas as medições, o filtro de Kalman se reduz à parte de predição.

In [42]:
ekf = EKF(F, G, init_state, init_sigma, Q, R, data)
for t in range(n_steps):
    ekf.predict(t)

In [43]:
ekf.Sigma

array([[ 3.07422492, -1.15359092,  3.75268398,  0.        ,  0.        ],
       [-1.15359092,  1.31431797, -1.3745781 ,  0.        ,  0.        ],
       [ 3.75268398, -1.3745781 ,  6.51041667,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , 10.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 10.        ]])

## Questão 05